In [10]:
import mitdeeplearning as mdl
import numpy as np
import cv2
from utils import load_data

In [ ]:
# Download the dataset
s = load_data()

# Print one of the samples
example = s[0]
print("Example of a sample:")
print(example)